In [1]:
%load_ext autoreload
%autoreload 2

import subprocess
import re
import os
from keypoint_classes import AngleCheck
from filters import *
from PIL import Image
import cv2

# BASE_DIR = os.path.dirname(os.path.realpath(__file__))
BASE_DIR = os.getcwd() # Может так?
img_root = os.path.join(BASE_DIR, 'media/good_img')
# img_root = os.path.join(BASE_DIR, 'media/img_from_video/user1')

# Dima dont touch! ONLY COMMENT!!!!!  :))
# mmpose_dir = '/home/kirill/PycharmProjects/MMproject/mmpose'
# my_dir = '/home/kirill/PycharmProjects/HandstandProject/'

mmpose_dir = '/home/dmitriy/mmcv/mmpose'
my_dir = '/home/dmitriy/MMProject-HandStand_and_other-'

# f_root = os.path.join(BASE_DIR, 'media/filtered')
# img_out = os.path.join(BASE_DIR, 'media/out_img')

f_root = os.path.join(my_dir, 'media/filtered')
img_out = os.path.join(my_dir, 'media/out_img')

img = 'tst_img1.png'
array_of_results = []


def get_img_rezult(mmpose_root, img_name, img_root_dir, f_root_dir, img_out_dir):
    rez = {}
    # filter to test
    ftypes = {99: 'NO', 0: 'CLAHE', 1: 'GRAY', 2: 'MEDIANBLUR_NG', 3: 'HSV', 4: 'SHARPEN', 5: 'SHARPEN2',
              6: 'SHARPEN2_NG', 7: 'EMBOSS', 8: 'VLINE', 9: 'AVERAGE', 10: 'AVERAGE_NG', 11: 'GAUSSIAN_BLUR',
              12: 'GAUSSIAN_BLUR_NG', 13: 'BILATERAL', 14: 'BILATERAL_NG', 15: 'AVERAGE_SHARP',
              16: 'AVERAGE_SHARP_BILATERAL', 17: 'MEDIANBLUR', 18: 'TRUNC', 19: 'PURE_BC', 20: 'BC_TRUNC',
              21: 'TRUNC_NG', 22: 'WB', 23: 'WB_TRUNC', 24: 'WB_TRUNC_NG', 25: 'THRESH_TOZERO',
              26: 'THRESH_TOZERO_NG', 27: 'GB_TRUNC', 28: 'GB_WB_TRUNC', 29: 'SH2_TRUNC', 30: 'SH2_MB_TRUNC'}
    ft = 99
    ### for tst_img5 - 18, 11, 9 and 5 made a good job
    ### for the
    resize = 500  # Set False to prevent resizing
    smooth = 3  # an additional numerical parameter like the smooth factor in the MEDIANBLUR
    alpha = 1.0  # Simple contrast control
    beta = 0  # Simple brightness control

    img_new = resized_and_filtered([mmpose_root, img_root_dir, img_name, f_root_dir], ftypes[ft],
                                   resize, smooth, alpha, beta)

    thresholds = {'strict': [0.150, 0.032, 0.018, 0.027, 0.026, 0.015],  # Median
                  'conservative': [0.2, 0.06, 0.034, 0.048, 0.064, 0.031],  # Median + Standard deviation
                  'week': [0.323, 0.088, 0.049, 0.069, 0.101, 0.047]}  # Median + 2xStandard deviation

    # Previous thresholds' values
    # ac_thr_wes = 0.150 ac_thr_wsh = 0.037 ac_thr_wsk = 0.019 ac_thr_shk = 0.026
    # ac_thr_hka = 0.027 ac_thr_wha = 0.018

    ac_thr_wes, ac_thr_wsh, ac_thr_wsk, ac_thr_shk, ac_thr_hka, ac_thr_wha = thresholds['week']

    # retrieve data from mmpose
    mmpose_out_dataset_str = subprocess.check_output(["./subscript", mmpose_dir, f_root_dir, img_new, img_out_dir])

    string_mm = mmpose_out_dataset_str.decode()
    arr_of_strings = string_mm.replace("\n", "").replace(" ", "").split("}")
    re_rule_f = '\'keypoints\':array\(.*?\)'
    re_rule_s = '\[(.+?)\]'
    arr_of_keypoint_str = []
    data_data = []
    for x in arr_of_strings:
        arr_of_keypoint_str.append(re.findall(re_rule_f, x))
    if arr_of_keypoint_str:
        for y in arr_of_keypoint_str:
            if y:
                substring_arr = re.findall(re_rule_s, y[0])
                substring_arr[0] = substring_arr[0].replace('[', '')
                s_ar = [list(map(float, (x.split(',')))) for x in substring_arr]
                data_data.append(s_ar)
        for data in data_data:
            result = AngleCheck(data, False)
            result.calculate([ac_thr_wes, ac_thr_wsh, ac_thr_wsk,
                              ac_thr_shk, ac_thr_hka, ac_thr_wha])
            if result.flag:
                rez = result.dict_of_angles
                break
        img_in = Image.open(f'{img_root_dir}/{img_name}')
        img_in_f = Image.open(f'{f_root_dir}/{img_new}')
        img_out = Image.open(f'{img_out_dir}/vis_{img_new}')
        img_in.show()
        img_in_f.show()
        img_out.show()
        return rez
    else:
        print(string_mm)
        return rez


d = get_img_rezult(mmpose_root=mmpose_dir, 
                   img_name=img, 
                   img_root_dir=img_root, 
                   f_root_dir=f_root, 
                   img_out_dir=img_out)

print(f"++++++{d}")

## For all img of user

# path, dirs, files = next(os.walk(img_root))
# file_count = len(files)
# for i in range(file_count):
#     imgname = f"img_{i}.png"
#     array_of_results.append(
#         get_img_rezult(img_name=imgname, img_root_dir=img_root, f_root_dir=f_root, img_out_dir=img_out))
#
# print(array_of_results)


/home/dmitriy/mmcv/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '
/home/dmitriy/mmcv/mmdetection/mmdet/core/anchor/anchor_generator.py:323: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/home/dmitriy/mmcv/mmdetection/mmdet/core/anchor/anchor_generator.py:360: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '


WES angle (True, 0.74)
WSH angle (True, 8.97)
WSK angle (True, 5.71)
SHK angle (True, 1.14)
HKA angle (True, 5.72)
WHA angle (True, 2.94)
++++++{'WES angle': [179.25726381426162, (True, 0.74)], 'WSH angle': [171.0269170815629, (True, 8.97)], 'WSK angle': [174.28882793544878, (True, 5.71)], 'SHK angle': [178.85602780814992, (True, 1.14)], 'HKA angle': [174.28411907070267, (True, 5.72)], 'WHA angle': [177.0640937558074, (True, 2.94)]}


In [90]:
import cv2
import os
from PIL import ImageFilter, Image
import numpy as np
from varname import nameof


def resize_img(img, max_image_height):
    scale_pct = float(max_image_height / img.shape[0])
    width = int(img.shape[1] * scale_pct)
    height = int(img.shape[0] * scale_pct)
    dsize = (width, height)
    
    return cv2.resize(img, dsize)

class Filtering:
    def __init__(self, img, ftype, path, smooth=3, alpha=1.0, beta=0):
        self.img = img
        self.ftype = ftype
        self.kernels = [np.array([[0, -1, 0],
                                  [-1, 5, -1],
                                  [0, -1, 0]]),
                        np.array([[-2, -1, 0],
                                  [-1, 1, 1],
                                  [0, 1, 2]]),
                        np.ones((5, 5), np.float32) / 25]
        self.path = path
        self.smooth = smooth
        self.a = alpha
        self.b = beta

    def extract_lines(self, write=False, vline=True):
        """
            Extracts vertical/horizontal lines from the input image
        """
        # Get rid of JPG artifacts
        img = cv2.threshold(self.img, 128, 255, cv2.THRESH_BINARY)[1]

        # Create structuring elements
        horizontal_size = 11
        vertical_size = 11
        horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontal_size, 1))
        verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, vertical_size))

        # Morphological opening
        mask1 = cv2.morphologyEx(self.img, cv2.MORPH_OPEN, horizontalStructure)
        mask2 = cv2.morphologyEx(self.img, cv2.MORPH_OPEN, verticalStructure)

        if write:
            cv2.imwrite(os.path.join(self.path[0], self.path[3], '{}_{}'.format(nameof(mask1), self.path[2])), mask1)
            cv2.imwrite(os.path.join(self.path[0], self.path[3], '{}_{}'.format(nameof(mask2), self.path[2])), mask2)

        if vline:
            return mask2, '{}_{}'.format(nameof(mask2), self.path[2])
        else:
            return mask1, '{}_{}'.format(nameof(mask1), self.path[2])

    def brightness_contrast_adj(self):
        new_image = np.zeros(self.img.shape, self.img.dtype)
        for y in range(self.img.shape[0]):
            for x in range(self.img.shape[1]):
                for c in range(self.img.shape[2]):
                    new_image[y, x, c] = np.clip(self.a * self.img[y, x, c] + self.b, 0, 255)
        return new_image

    def wb(self):
        result = cv2.cvtColor(self.img, cv2.COLOR_BGR2LAB)
        avg_a = np.average(result[:, :, 1])
        avg_b = np.average(result[:, :, 2])
        for x in range(result.shape[0]):
            for y in range(result.shape[1]):
                l, a, b = result[x, y, :]
                # fix for CV correction
                l *= 100 / 255.0
                result[x, y, 1] = a - ((avg_a - 128) * (l / 100.0) * 1.1)
                result[x, y, 2] = b - ((avg_b - 128) * (l / 100.0) * 1.1)
        result = cv2.cvtColor(result, cv2.COLOR_LAB2BGR)
        return result

    def clache(self):
        lab = cv2.cvtColor(self.img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        cl = clahe.apply(l)
        limg = cv2.merge((cl, a, b))
        return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    def gray(self):
        return cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)
    
    def bgr2rgb(self):
        return cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)

    def rgb(self):
        return cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)

    def medianb(self):
        return cv2.medianBlur(self.img, self.smooth)

    def sharpen(self):
        return cv2.filter2D(self.img, -1, self.kernels[0])

    def emboss(self):
        image = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)
        return cv2.filter2D(image, -1, self.kernels[1])

    def avg(self):
        image = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)
        return cv2.filter2D(image, -1, self.kernels[2])

    def bilateral(self):
        image = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)
        return cv2.bilateralFilter(image, 9, 75, 75)

    def trunc(self):
        image = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)
        _, final = cv2.threshold(image, 127, 255, cv2.THRESH_TRUNC)
        return final

    def threstozero(self):
        image = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)
        _, final = cv2.threshold(image, 127, 255, cv2.THRESH_TOZERO)
        return final

    def hcv(self):
        return cv2.cvtColor(self.img, cv2.COLOR_BGR2HSV)

    def gblur(self):
        image = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)
        return cv2.GaussianBlur(image, (5, 5), 0)

    def apply_filter(self):
        if self.ftype == 'CLAHE':
            res = self.clache()

        elif self.ftype == 'GRAY':
            res = self.gray()
            
        elif self.ftype == 'BGR2RGB':
            res = self.bgr2rgb()

        elif self.ftype == 'MEDIANBLUR':
            res = self.medianb()

        elif self.ftype == 'HSV':
            res = self.hcv()

        elif self.ftype == 'SHARPEN':
            res = self.sharpen()

        elif self.ftype == 'EMBOSS':
            res = self.emboss()

        elif self.ftype == 'VLINE':
            res = self.extract_lines(write=False, vline=True)[0]

        elif self.ftype == 'AVERAGE':
            res = self.avg()

        elif self.ftype == 'GAUSSIAN_BLUR':
            res = self.gblur()

        elif self.ftype == 'BILATERAL':
            res = self.bilateral()

        elif self.ftype == 'TRUNC':
            res = self.trunc()

        elif self.ftype == 'BC':
            res = self.brightness_contrast_adj()

        elif self.ftype == 'WB':
            res = self.wb()

        elif self.ftype == 'THRESH_TOZERO':
            res = self.threstozero()

        else:
            res = self.img

        return res

def resized_and_filtered(dirs, fltrs, max_image_height, smooth=3, alpha=1.0, beta=0):
    """
    - dirs is a list of directories in the followinf order:
        mmpose_dir, img_root, img, filtered_root
    - fltrs - list of filters
    - max_image_height - desirable image height
    - smooth - MEDIANBLUR smoothing factor
    - alpha, beta - brightness and contrast adjustable parameters
    """
    
    src = cv2.imread(os.path.join(dirs[0], dirs[1], dirs[2]), cv2.IMREAD_UNCHANGED)
    
    ### Resize
    if max_image_height:
        output = resize_img(src, max_image_height)
    else:
        scale_pct = 'no'
        output = src
    ###
    
    output = Filtering(output, fltrs[0], dirs).apply_filter()
    for i in range(1, len(fltrs)):
        output = Filtering(output, fltrs[i], dirs).apply_filter()
#         cv2.imshow(str(i), new)
#         cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
    out_name = '{}_{}_{}.png'.format(dirs[2].split('.')[0], scale_pct, "_".join(fltrs))
    if not cv2.imwrite(os.path.join(dirs[0], dirs[3], '{}'.format(out_name)), final):
        raise Exception("Could not write image")
#     else:
#         cv2.imwrite(os.path.join(dirs[0], dirs[3], '{}'.format(out_name)), final)
    return output

In [91]:
fltrs = {0:'GRAY', 1:'WB', 2:'BC', 3:'HSV', 4:'SHARPEN', 5:'MEDIANBLUR', 
         6:'AVERAGE', 7:'GAUSSIAN_BLUR', 8:'TRUNC', 9:'CLAHE', 10:'BILATERAL',
         11:'THRESH_TOZERO', 12:'VLINE', 13:'EMBOSS', 14:'BGR2RGB'}

"NOTE: WB is not applicable for gray scale image! Please, keep appropriate order of filters."

apply_following = [fltrs[1], fltrs[2], fltrs[4],  fltrs[5]]

In [88]:
"_".join(apply_following)

'WB_BC_SHARPEN_MEDIANBLUR'

In [29]:
paths = [mmpose_dir, img_root, img, f_root]

In [20]:
pctr = resized_and_filtered(paths, c_ftype, 500)

In [93]:
cv2.imshow("image", pctr)
cv2.waitKey(0)
new = Filtering(pctr, apply_following[0], paths).apply_filter()
cv2.imshow("image", new)
cv2.waitKey(0)
for i in range(1, len(apply_following)):
    new = Filtering(new, apply_following[i], paths).apply_filter()
    cv2.imshow(str(i), new)
    cv2.waitKey(0)

cv2.destroyAllWindows()

In [85]:
cv2.imshow("image", pctr)
cv2.waitKey(0)
cv2.imshow("image1", Filtering(pctr, apply_following[0], paths).apply_filter())
cv2.waitKey(0)
cv2.imshow("image2", new)
cv2.waitKey(0)
cv2.destroyAllWindows()